<a href="https://colab.research.google.com/github/Wang-Kevin900/DataVisualization/blob/main/EsercizioReti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the various library needed to the program, importing the txt sheet where all the data are (format: date - hour - value) and lastly creating all the needed variable

In [ ]:
from google.colab import files
from datetime import datetime

file_data = files.upload()
valori = list(file_data.values());

date_list = []
value_list = []
date_format = 10
hour_format = 8

string_of_data = ""

Rearrange all the data in sets of tuples 

In [ ]:
for i in valori[0]:
  if i == 13:
    pass
  elif i != 10:
    string_of_data = string_of_data + chr(i);
  else:
    date_list.append(string_of_data[0:date_format+1+hour_format])
    value_list.append(string_of_data[date_format+1+hour_format+1:])
    string_of_data = ""

Convert the value from string to float, also replace the "," in "." so that the casting is possible

In [ ]:
list_of_float_value = []
for item in value_list:
    value = item.replace(",", ".")
    list_of_float_value.append(float(value))

value_list = list_of_float_value

Convert the value from string to datetime format


In [ ]:
#from datetime import datetime

#list_of_date_value = []
#for item in date_list:
#    list_of_date_value.append(datetime.strptime(item, '%d-%m-%Y %H:%M:%S'))

#date_list = list_of_date_value

Convert all the lists (date list and value list) in Pandas DataFrame, then, visualize the list created

In [ ]:
import pandas as pd

data = pd.DataFrame({'date': date_list,
                     'value': value_list})

data

Draw everything in a chart using the altair library

In [ ]:
import altair as alt

brush = alt.selection_interval(encodings=['x'])

color = alt.condition(brush, alt.value('blue'), alt.value('lightgray'))

chart = alt.Chart(data).mark_line(point={
    "filled": False,
    "fill": 'white'
    }).encode(
        y=alt.Y('value', sort=alt.EncodingSortField('value', order='descending')),
        x='date',
        color=color
).add_selection(
    brush
)

chart 

In [ ]:
import json

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

df = pd.DataFrame({
    "x": [1,2,1,2],
    "y": [1,2,3,4],
    "customdata": [1,2,3,4],
    "fruit": ["apple", "apple", "orange", "orange"]
})

fig = px.scatter(df, x="x", y="y", color="fruit", custom_data=["customdata"])

fig.update_layout(clickmode='event+select')

fig.update_traces(marker_size=20)

app.layout = html.Div([
    dcc.Graph(
        id='basic-interactions',
        figure=fig
    ),

    html.Div(className='row', children=[
        html.Div([
            dcc.Markdown("""
                **Hover Data**

                Mouse over values in the graph.
            """),
            html.Pre(id='hover-data', style=styles['pre'])
        ], className='three columns'),

        html.Div([
            dcc.Markdown("""
                **Click Data**

                Click on points in the graph.
            """),
            html.Pre(id='click-data', style=styles['pre']),
        ], className='three columns'),

        html.Div([
            dcc.Markdown("""
                **Selection Data**

                Choose the lasso or rectangle tool in the graph's menu
                bar and then select points in the graph.

                Note that if `layout.clickmode = 'event+select'`, selection data also
                accumulates (or un-accumulates) selected data if you hold down the shift
                button while clicking.
            """),
            html.Pre(id='selected-data', style=styles['pre']),
        ], className='three columns'),

        html.Div([
            dcc.Markdown("""
                **Zoom and Relayout Data**

                Click and drag on the graph to zoom or click on the zoom
                buttons in the graph's menu bar.
                Clicking on legend items will also fire
                this event.
            """),
            html.Pre(id='relayout-data', style=styles['pre']),
        ], className='three columns')
    ])
])


@app.callback(
    Output('hover-data', 'children'),
    Input('basic-interactions', 'hoverData'))
def display_hover_data(hoverData):
    return json.dumps(hoverData, indent=2)


@app.callback(
    Output('click-data', 'children'),
    Input('basic-interactions', 'clickData'))
def display_click_data(clickData):
    return json.dumps(clickData, indent=2)


@app.callback(
    Output('selected-data', 'children'),
    Input('basic-interactions', 'selectedData'))
def display_selected_data(selectedData):
    return json.dumps(selectedData, indent=2)


@app.callback(
    Output('relayout-data', 'children'),
    Input('basic-interactions', 'relayoutData'))
def display_relayout_data(relayoutData):
    return json.dumps(relayoutData, indent=2)


if __name__ == '__main__':
  app.run_server(mode='inline',host="0.0.0.0",port=1005)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scatter(x=date_list, y=list_of_float_value, mode="markers+lines"))

scatter = fig.data[0]

def update_point(trace, points, selector):
  print("Sleeping")
scatter.on_click(update_point)

fig.show(renderer="colab")

In [ ]:
print(scatter.selectedpoints)